In [140]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import namedtuple
import os

In [141]:
Data = namedtuple('Data', ['name', 'all_coverage_filepath', 'browser_coverage_filepath', 'exectime', 'color', 'label'])

def map_data(data: Data):
    seconds_per_hour = 3600
    cov = None
    with open(data.browser_coverage_filepath, 'r') as f:
        cov = pd.read_csv(f, header=None)

    num_edges = cov.iloc[0,1]
    cov.columns = ['input', 'browser_coverage']
    cov.drop(cov.index[0], inplace=True)

    with open(data.all_coverage_filepath, 'r') as f:
        all_cov = pd.read_csv(f, header=None)
        all_cov.columns = ['input', 'all_coverage']
        all_cov.drop(all_cov.index[0], inplace=True)
        cov = pd.concat([cov, all_cov['all_coverage']], axis=1)

    cov['input'] = cov['input'].astype(float).apply(lambda x: x*data.exectime / seconds_per_hour)
    cov.rename(columns={'input': 'time'}, inplace=True)

    cov = cov[cov['time'] <= 24]

    cov['browser_coverage'] = cov['browser_coverage'].astype(float).apply(lambda x: x/num_edges * 100)
    cov['all_coverage'] = cov['all_coverage'].astype(float).apply(lambda x: x/num_edges * 100)
    return cov


def plot_coverage(plots: list[tuple[list[Data],str]], filename: str|None = None):
    # Create a plot
    plt.rcParams.update({'font.size': 22})
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(13, 6))

    linewidth = 2.5

    for i, plot in enumerate(plots):
        data, title = plot
        ax = axes[i]

        total_max = 0

        for d in data:
            cov = map_data(d)
            ax.plot(cov['time'], cov['all_coverage'], label=f'{d.label} (all)' if i == len(plots) - 1 else '', color=d.color, linewidth=linewidth)
            ax.plot(cov['time'], cov['browser_coverage'], label=f'{d.label} (browser)' if i == len(plots) - 1 else '', color=d.color, linestyle='--', linewidth=linewidth)
            total_max = max(cov['all_coverage'].max(), total_max)

        ax.set_title(title)
        ax.set_ylabel('% Edge Coverage')
        ax.set_xlabel('Time (hours)')
        ax.set_xlim(-0.25, 24)  # Limit x-axis to 24 seconds
        ax.set_ylim(0, total_max + 1)
        ax.locator_params(axis="y", integer=True, tight=True)
        ax.locator_params(axis="x", tight=True)
        ax.grid()

    # plt.title(title)
    # plt.ylabel('% Edge Coverage')
    # plt.xlabel('Time (hours)')
    # plt.legend()
    # plt.grid()
    # plt.xlim(-0.25, 24)  # Limit x-axis to 24 seconds
    # plt.ylim(0, total_max + 1)  # Adjust y-axis for visibility
    # axes[1].legend(loc='upper center',fancybox=False, shadow=False, ncol=4)

    handles, labels = plt.gca().get_legend_handles_labels()
    order = [2,0,3,1]
    plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order],loc='lower right')
    plt.tight_layout()  # Adjust layout to prevent clipping of titles/labels

    if filename:
        fig.savefig(filename)
        os.system(f'pdfcrop {filename} {filename}')
    else:
        fig.show()
    plt.close()



In [142]:


orange = (239/256, 138/256, 23/256)
blue = (60/256, 145/256, 230/256)

# fuzzorigin old: different IP's
# fuzzorigin new: different ports

chrome_data = [
    # Data('IPCrafter Chrome', 'ipcrafter_chrome_all_coverage.csv', 'ipcrafter_chrome_browser_coverage.csv', 4.0, 'green', 'ipcrafter'),
    Data('Fuzzorigin Chrome', 'fuzzorigin_chrome_all_coverage_new.csv', 'fuzzorigin_chrome_browser_coverage_new.csv', 7.67, blue, 'fuzzorigin'),
    Data('IPCrafter Chrome', 'ipcrafter_chrome_all_coverage_new.csv', 'ipcrafter_chrome_browser_coverage_new.csv', 3.02,  orange, 'our fuzzer'),
]

firefox_data = [
    # Data('IPCrafter Firefox', 'ipcrafter_firefox_all_coverage.csv', 'ipcrafter_firefox_browser_coverage.csv', 7.15, 'green', 'ipcrafter'),
    Data('Fuzzorigin Firefox', 'fuzzorigin_firefox_all_coverage_new.csv', 'fuzzorigin_firefox_browser_coverage_new.csv', 7.12, blue, 'fuzzorigin'),
    Data('IPCrafter Firefox', 'ipcrafter_firefox_all_coverage_new.csv', 'ipcrafter_firefox_browser_coverage_new.csv', 6.67,  orange, 'our fuzzer'),
]
plot_coverage([(chrome_data, 'Chrome Coverage'),(firefox_data, 'Firefox Coverage')], 'coverage.pdf')


sh: 1: pdfcrop: not found
